In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [ ]:
import zipfile
from pathlib import Path
from fastai import *
from fastai.vision.all import *
import torch
Data= Path("../input/aerial-cactus-identification/")
test_df=pd.read_csv("../input/aerial-cactus-identification/sample_submission.csv")
train_df=pd.read_csv("../input/aerial-cactus-identification/train.csv")
with zipfile.ZipFile(Data/"train.zip","r") as z:
    z.extractall("../kaggle/temp/")
    
with zipfile.ZipFile(Data/"test.zip","r") as z:
    z.extractall("../kaggle/temp/")

In [ ]:
test_img = ImageDataLoaders.from_df(test_df, path='../kaggle/temp', folder='test')
#trfm = aug_transforms(size=224,do_flip=True, flip_vert=True, max_rotate=10.0, max_zoom=1.1, max_lighting=0.2, max_warp=0.2, p_affine=0.75, p_lighting=0.75)
train_img = ImageDataLoaders.from_df(train_df,path='../kaggle/temp', folder='train',test='test_img')        

In [ ]:
learn = cnn_learner(train_img, resnet18, metrics=[error_rate, accuracy],loss_fuction=CrossEntropyLossFlat())


In [ ]:
doc(learn.get_preds)

In [ ]:
learn.fine_tune(1)
learn.fit_one_cycle(5, slice(0.003))

In [ ]:
preds,_ = learn.get_preds(dl=train_img.test_dl(get_image_files('../kaggle/temp/test/'),shuffle=False,drop_last=False))
#test_df.has_cactus = preds.numpy()[:, 0]

In [ ]:
#pre=[]
names=[]
fnames = get_image_files('../kaggle/temp/test/')
for x in fnames:
    names.append(x.name)
submission_df = pd.DataFrame(data={'id':names,'has_cactus':preds.numpy()[:,0] })

In [ ]:

submission_df.to_csv('/kaggle/working/submission.csv',index=False)
submission_df.head()